In [6]:
search_for = 109
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.012993812561035156
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 142465181
type: [1, 1, 1, 1, 109] 109
cases of this type: 141158161
10000 17.659317606702917
20000 19.718606126396157
30000 17.825828706011155
40000 49.99473104165639
50000 94.10542294783605
60000 111.8838146319805
70000 127.28952643220774
80000 93.37672762301632
90000 106.0757535546141
100000 125.50030755282478
110000 116.16152006275249
120000 100.82222644539662
130000 108.83894624111147
140000 122.1887925809083
150000 112.25771634459642
160000 107.35038815494923
170000 93.9958832721619
180000 98.68592189868507
190000 98.0159351882855
200000 100.32050014637223
210000 92.90770291891769
220000 92.36217269089977
230000 101.47301782340688
240000 92.49900346692863
250000 99.91047819279014
260000 98.31317560776641
270000 95.03144548984471
280000 101.93125173341625
290000 94.74339503683059
300000 101.1037169168172
310000 95.68144120502

3080000 87.52979520767718
3090000 88.39004001621745
3100000 94.71958708720327
3110000 90.2250878666194
3120000 87.64232727002742
3130000 90.04057466512637
3140000 89.87314685949676
3150000 85.72207200559598
3160000 95.55858849242824
3170000 90.05301504622552
3180000 84.95315037419834
3190000 99.52497070493276
3200000 109.56982408773108
3210000 92.36552648807489
3220000 12.930048835084902
3230000 12.434881514232476
3240000 12.474339266479612
3250000 13.071636833506822
3260000 12.509430533351878
3270000 44.00444031696079
3280000 129.57466629495957
3290000 114.84136099688293
3300000 87.08835464870518
3310000 88.04803836649899
3320000 95.7632140020989
3330000 87.33563102790478
3340000 85.86645901410805
3350000 95.40125468769907
3360000 89.15050712812467
3370000 84.81015446960149
3380000 92.00546857067765
3390000 90.09254421498123
3400000 82.49293876348501
3410000 90.94702971704588
3420000 92.32208577822945
3430000 83.19987927747412
3440000 86.77787904419358
3450000 85.77505551580576
346000

6230000 92.19957194278004
6240000 88.64931078260513
6250000 91.307776490455
6260000 92.2135901580965
6270000 88.94143116020497
6280000 90.07064289129069
6290000 91.32654331544049
6300000 90.13380013151873
6310000 103.1955508794833
6320000 93.9944207693338
6330000 91.96865370639287
6340000 88.63936685705802
6350000 94.41742851496339
6360000 91.04636667990343
6370000 85.60372303660775
6380000 97.42233525115729
6390000 108.3452657552677
6400000 42.58731357089685
6410000 30.671385997234104
6420000 31.36176177201701
6430000 30.197193382899712
6440000 30.247101791545273
6450000 30.84652406833467
6460000 111.41746269137407
6470000 104.25580428351688
6480000 102.13220271936565
6490000 92.2869942758283
6500000 97.57519552811557
6510000 118.62428303483482
6520000 25.085596900858544
6530000 24.15134370187205
6540000 24.74904324656647
6550000 24.246333946117165
6560000 24.69584322941198
6570000 24.72864002326529
6580000 112.04120924628334
6590000 97.52067756929287
6600000 94.39814869483988
6610000

9390000 86.07015327403074
9400000 89.48484063479837
9410000 85.53232652001161
9420000 87.74028292693592
9430000 87.32620096540919
9440000 84.74064414206127
9450000 89.67135135892747
9460000 91.90841110805594
9470000 81.3120665653874
9480000 86.65954372630206
9490000 84.06714668077504
9500000 85.58177535808379
9510000 87.70758793189539
9520000 85.71635513667844
9530000 82.29074343765724
9540000 86.18461347248432
9550000 89.9461962338692
9560000 84.62152897768279
9570000 93.27803865268915
9580000 88.5631968754811
9590000 81.02125048153586
9600000 87.06561087602242
9610000 85.70955781599281
9620000 82.58588229191096
9630000 83.50418454148516
9640000 87.65222228603342
9650000 75.13825654912613
9660000 75.70880651858847
9670000 86.40854107980681
9680000 95.85878466796565
9690000 35.994956467579996
9700000 11.607659815346718
9710000 11.47612015021821
9720000 11.651161255045594
9730000 11.830896465007848
9740000 11.783296627175014
9750000 83.16103939078049
9760000 97.82303447796644
9770000 79

12450000 83.06644709105781
12460000 79.71872353936375
12470000 79.63449801322079
12480000 80.49215164475108
12490000 80.20886008602135
12500000 80.60549126027033
12510000 86.2297411676789
12520000 86.70204730241956
12530000 91.17070942931338
12540000 88.39689935616366
12550000 85.35826768812814
12560000 87.64443455635268
12570000 84.04278879114358
12580000 87.50823921925209
12590000 88.91725066993307
12600000 79.2396938770192
12610000 81.10325980887744
12620000 84.42584525882022
12630000 84.30009894064735
12640000 83.67508875592304
12650000 84.15470762929691
12660000 81.19751171005025
12670000 86.48413553088109
12680000 87.76322762692872
12690000 93.5673600625748
12700000 85.6984775951831
12710000 89.63132025932633
12720000 83.27591089134258
12730000 86.62647347076556
12740000 83.94217546577698
12750000 83.5221270666536
12760000 87.57591841190006
12770000 90.71878714934905
12780000 81.26602771362792
12790000 87.19883355202823
12800000 92.28327639017449
12810000 78.23060009340435
128200

...

128790000 7.213779612798532
128800000 7.575187816103327
128810000 7.159429743953625
128820000 8.581598262413694
128830000 6.108723188459409
128840000 1.1883542664300202
128850000 1.1082114025273258
128860000 1.168801219348477
128870000 1.13337266278551
128880000 1.1463071923896868
128890000 6.088232373974879
128900000 9.372498184936106
128910000 8.547200443028284
128920000 6.483795328643501
128930000 6.823321188080371
128940000 6.720131116744783
128950000 7.586415782694373
128960000 7.513272393873474
128970000 7.86975809399728
128980000 7.852578357474931
128990000 7.858844756896278
129000000 7.616313880503907
129010000 7.9520188410355575
129020000 7.316093572802127
129030000 7.646121966757417
129040000 7.540745540592611
129050000 8.224736488619982
129060000 7.5379356914387605
129070000 8.314984132437123
129080000 8.604441213779788
129090000 8.027331133467072
129100000 8.025877060503124
129110000 8.158096026007652
129120000 8.229965171463304
129130000 8.206212547227997
129140000 7.63446

131710000 6.5054113434611365
131720000 6.452399362708688
131730000 6.134192978024125
131740000 5.738969449450837
131750000 7.004698546182037
131760000 5.910024607407351
131770000 6.109375386704459
131780000 6.807105722790957
131790000 6.512276060322225
131800000 6.247215628473122
131810000 5.342149845769226
131820000 5.5494411925235125
131830000 6.172043353577746
131840000 6.542860605932196
131850000 5.097883881708211
131860000 6.261591326732099
131870000 5.592511905445396
131880000 5.13703332269409
131890000 5.89574568096092
131900000 5.63680040303278
131910000 5.976466982456207
131920000 6.163558791589208
131930000 6.210053999066631
131940000 4.888115232459207
131950000 5.820823553341197
131960000 7.1635165754451755
131970000 6.82689482931455
131980000 4.251615777964645
131990000 1.787189125908971
132000000 1.7113526689395508
132010000 1.7202864743512072
132020000 1.6085523139339089
132030000 1.6252815466538602
132040000 5.313112356364978
132050000 7.722157906984091
132060000 6.20895

134630000 1.2086345211822787
134640000 0.9526619190950856
134650000 2.2241207504485714
134660000 5.209421234833101
134670000 5.349208513589428
134680000 4.475059814760725
134690000 4.548715371713877
134700000 5.848685482992079
134710000 3.9671053884919814
134720000 1.3238324541202189
134730000 1.5701572867487867
134740000 1.2166848663038348
134750000 1.037418126873195
134760000 1.3584512913747295
134770000 3.225718890414761
134780000 5.130217847395003
134790000 4.998805481805947
134800000 4.802443857488692
134810000 4.102565088807344
134820000 4.2949080282701715
134830000 4.555503673872934
134840000 3.927768019241452
134850000 4.331303276602341
134860000 4.752065930933601
134870000 4.188730771609426
134880000 4.100874985813956
134890000 4.284550702394412
134900000 3.9663208300493955
134910000 3.909122699939834
134920000 4.180699570154701
134930000 4.351539323055248
134940000 4.038613862686296
134950000 4.1790216703229675
134960000 4.738695252794146
134970000 4.3428191337113375
13498000

137510000 2.5723088059963186
137520000 2.8714256936235425
137530000 2.8951144115313756
137540000 2.8695410742182137
137550000 2.721601979443868
137560000 2.444035350487636
137570000 2.926309780349136
137580000 2.803535287006643
137590000 3.126180601637516
137600000 2.5850041320842903
137610000 2.973814176128937
137620000 2.878288613933868
137630000 2.9191701299999755
137640000 2.861789021571696
137650000 2.8076918649968308
137660000 2.4319949599046113
137670000 2.66044952965294
137680000 2.8185970731679664
137690000 2.6355019430592654
137700000 2.830774187992215
137710000 2.323336823371073
137720000 2.500801381626467
137730000 2.7149315763772
137740000 2.771368611910019
137750000 2.3463481328191755
137760000 2.432739180238644
137770000 2.43396615948793
137780000 2.253720996907075
137790000 2.7426564194701775
137800000 2.6975109212599793
137810000 2.262697693662485
137820000 2.3644985728134316
137830000 2.4773626497751144
137840000 2.3605602871693967
137850000 2.6410324458755188
1378600

140380000 1.1779289540563889
140390000 1.1497258629237215
140400000 1.0099967180491554
140410000 1.1801005168226162
140420000 1.1672211568906505
140430000 1.040757889305194
140440000 1.1168381037155921
140450000 1.1709428447362782
140460000 1.071386964950237
140470000 1.190025438569877
140480000 1.33536303943336
140490000 0.13825016341533927
140500000 0.15060901842901442
140510000 0.12223087336262067
140520000 0.15218198988032342
140530000 0.15589573745675883
140540000 0.16223346100491284
140550000 1.3627383597559928
140560000 1.1490807929492792
140570000 1.1116640971769491
140580000 1.0742887603914737
140590000 0.9414466097542776
140600000 0.9384793206388354
140610000 1.1097690468921728
140620000 0.9509427042734027
140630000 0.9210631797953248
140640000 1.044121435774843
140650000 0.9432591354419787
140660000 0.9015770541897217
140670000 0.9867299278806447
140680000 0.7858178029088047
140690000 1.0213117441824673
140700000 1.0801770867027243
140710000 1.0240596677351064
140720000 0.91